In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as slin
import warnings
warnings.filterwarnings('ignore')

In [2]:
TPM = np.array([
    [0.8, 0.1, 0.1, 0],
    [0, 0.7, 0.2, 0.1],
    [0, 0, 0.7, 0.3],
    [0, 0, 0, 1]
])


In [3]:
state = [0,1,2,3]
action = [0,1,2]

imm_reward = np.array([
    [20000],
    [16000],
    [12000],
    [5000]
]).reshape(-1)

action_cost = np.array([
    [0],
    [2000],
    [10000]
])

policy_state_transition = [(0,0,0), (1,0,1), (1,2,0), (2,0,2), (2,1,1), (2,2,0), (3,0,3), (3,1,1), (3,2,0)]


# Create the immediate reward matrix for states and actions

In [4]:
def state_action_imm_rew(policy_state_transition, imm_reward, action_cost):

    state_action = np.zeros((len(state),len(action),1))
    state_action_imm_rew = np.array(state_action.copy())
    state_action_imm_rew
    for s, a, s1 in policy_state_transition:
        if s == 0:
            state_action_imm_rew[s,a,:] = imm_reward[s1] - action_cost[a]
        elif (s == 1) & (a == 1):
            state_action_imm_rew[s,a,:] = 0
        elif (s == 1) & (a == 2):
            state_action_imm_rew[s,a,:] = imm_reward[s1]- action_cost[a]
        else:
            state_action_imm_rew[s,a,:] = imm_reward[s1] - action_cost[a]
    return state_action_imm_rew
        
        
state_action_imm_rew = state_action_imm_rew(policy_state_transition, imm_reward, action_cost)
state_action_imm_rew            

array([[[20000.],
        [    0.],
        [    0.]],

       [[16000.],
        [    0.],
        [10000.]],

       [[12000.],
        [14000.],
        [10000.]],

       [[ 5000.],
        [14000.],
        [10000.]]])

# Inputs to test Policy and reward -- Having challenge to incorporate for period for discount factoring

# New Transition Matrix based on Actions

In [5]:
def get_value_function(TPM, policy_state, reward):
    transitions = []
    rewards = []
    for p in policy_state:
        transitions.append(TPM[p[2]])
        rewards.append(state_action_imm_rew[p[0],p[1]])
    new_mat =np.array(transitions) * discount ** (1)
    #for i in np.arange(future_period):
    #    if i > 0:
    #        new_mat += np.array(transitions) * discount ** (i+1)
    PP1 = new_mat * -1 + np.identity(TPM.shape[0])
    print(rewards)
    value_fn = slin.solve(PP1, rewards)
    return value_fn

In [6]:
discount = 0.95
future_period = 1 # not yet ready to solve for infinite sequence
policy = [0,0,1,1] # what policy we are defining # read from source or input entered by hand
policy_state = [(0,0,0), (1,0,1), (2,1,1), (3,1,1)] # this has to be created by hand 
#/ input should be read from source # Format: initial State, Action, Current State

value_fn = get_value_function(TPM, policy_state, state_action_imm_rew)

value_fn

[array([20000.]), array([16000.]), array([14000.]), array([14000.])]


array([[326850.],
       [308600.],
       [306600.],
       [306600.]])

# New Inputs -- Need to first test for only changed policy, then if new policy > than old policy, we recompute (Policy Iteration)

In [7]:
t_old = np.sum(value_fn)
t_old

1248649.9999999981

In [87]:
discount = 0.95
future_period = 1 # not yet ready to solve for infinite sequence
policy = [0,0,1,2] # what policy we are defining # read from source or input entered by hand
policy_state = [(0,0,0), (1,0,1), (2,1,1), (3,2,0)] # this has to be created by hand / input should be read from source

In [89]:
value_fn = get_value_function(TPM, policy_state,state_action_imm_rew)
value_fn

[array([20000.]), array([16000.]), array([14000.]), array([10000.])]


array([[337895.52238806],
       [322552.23880597],
       [320552.23880597],
       [327895.52238806]])

In [117]:
policy_state1 = [(0,0,0), (1,0,1), (2,1,1), (3,1,1)]
policy_state2 = [(0,0,0), (1,0,1), (2,1,1), (3,2,0)]

changed_policy=[policy_state1[i] == policy_state2[i] for i in range(len(policy_state1))]
changed_policy

[True, True, True, False]

In [126]:
changed_index = np.where(np.array(changed_policy)!=True)[0][0]
changed_index

3

In [136]:
value_fn1 = value_fn[0:changed_index:]
value_fn1

array([[326850.],
       [308600.],
       [306600.]])

# Checking for one policy and only the changed field, if the new value is higher we will then solve for new values, and this policy becomes the better policy

In [165]:
transitions = []
rewards = []
p,q,r = policy_state2[changed_index]

val_new = (state_action_imm_rew[p,q] + discount * TPM[r, list(set(state) - set([changed_index]))].dot(value_fn1)) / (1 - TPM[r, changed_index])
t_new = np.sum(value_fn1) + val_new
if t_new > t_old:
    print("New Policy is better. hence recomputing.")
    policy_state = [(0,0,0), (1,0,1), (2,1,1), (3,2,0)]
    value_fn = get_value_function(TPM, policy_state,state_action_imm_rew)
value_fn
t_old = np.sum(value_fn)

New Policy is better. hence recomputing.
[array([20000.]), array([16000.]), array([14000.]), array([10000.])]


# Trying to automate Policy iteration algorithm - WIP
- Not Full proof
- It looks at the whole state space, there should be a better way for checking to reduce iterations
- Need to find out total policy space
- Need variables which will contain each policy space

In [17]:
policy_state_transition = [[(0,0,0)], [(1,0,1), (1,2,0)], [(2,0,2), (2,1,1), (2,2,0)], [(3,0,3), (3,1,1), (3,2,0)]]

State 0 has only one Policy, nothing to optimize.


[1, 2, 3]

In [33]:
x1 = policy_state_transition[0]
x2 = policy_state_transition[1]
x3 = policy_state_transition[2]
x4 = policy_state_transition[3]

maxVal = 0
best_policy = []

for i in range(len(x1)):
    for j in range(len(x2)):
        for k in range(len(x3)):
            for l in range(len(x4)):
                policy = []

                policy.append(x1[i])
                policy.append(x2[j])
                policy.append(x3[k])
                policy.append(x4[l])
                print(policy)
                value_fn = get_value_function(TPM, policy,state_action_imm_rew)
                if (sum(value_fn) > maxVal):
                    maxVal = sum(value_fn)
                    best_policy = policy

[(0, 0, 0), (1, 0, 1), (2, 0, 2), (3, 0, 3)]
[array([20000.]), array([16000.]), array([12000.]), array([5000.])]
[(0, 0, 0), (1, 0, 1), (2, 0, 2), (3, 1, 1)]
[array([20000.]), array([16000.]), array([12000.]), array([14000.])]
[(0, 0, 0), (1, 0, 1), (2, 0, 2), (3, 2, 0)]
[array([20000.]), array([16000.]), array([12000.]), array([10000.])]
[(0, 0, 0), (1, 0, 1), (2, 1, 1), (3, 0, 3)]
[array([20000.]), array([16000.]), array([14000.]), array([5000.])]
[(0, 0, 0), (1, 0, 1), (2, 1, 1), (3, 1, 1)]
[array([20000.]), array([16000.]), array([14000.]), array([14000.])]
[(0, 0, 0), (1, 0, 1), (2, 1, 1), (3, 2, 0)]
[array([20000.]), array([16000.]), array([14000.]), array([10000.])]
[(0, 0, 0), (1, 0, 1), (2, 2, 0), (3, 0, 3)]
[array([20000.]), array([16000.]), array([10000.]), array([5000.])]
[(0, 0, 0), (1, 0, 1), (2, 2, 0), (3, 1, 1)]
[array([20000.]), array([16000.]), array([10000.]), array([14000.])]
[(0, 0, 0), (1, 0, 1), (2, 2, 0), (3, 2, 0)]
[array([20000.]), array([16000.]), array([1000

In [34]:
best_policy

[(0, 0, 0), (1, 2, 0), (2, 2, 0), (3, 2, 0)]

# LP formulation for MDP -- Need to generalize the equation formation

In [180]:
from pulp import *
# initialize the model
prob = LpProblem("mdp01", LpMinimize)

#########
policy_state_transition = [(0,0,0), (1,0,1), (1,2,0), (2,0,2), (2,1,1), (2,2,0), (3,0,3), (3,1,1), (3,2,0)]

transitions = []
rewards = []
for p in policy_state_transition:
    transitions.append(TPM[p[2]])
    rewards.append(state_action_imm_rew[p[0],p[1]])
new_mat =np.array(transitions) * discount ** (1)

T = len(state)
# ---------------------
# VARIABLES
# ---------------------

dv = LpVariable.dicts("dv", range(0, T), 0, None)

# Constraints

prob += dv[0] >= lpSum([new_mat[0, i] * dv[i] for i in np.arange(T)]) + rewards[0]

prob += dv[1] >= lpSum([new_mat[1, i] * dv[i] for i in np.arange(T)]) + rewards[1]
prob += dv[1] >= lpSum([new_mat[2, i] * dv[i] for i in np.arange(T)]) + rewards[2]

prob += dv[2] >= lpSum([new_mat[3, i] * dv[i] for i in np.arange(T)]) + rewards[3]
prob += dv[2] >= lpSum([new_mat[4, i] * dv[i] for i in np.arange(T)]) + rewards[4]
prob += dv[2] >= lpSum([new_mat[5, i] * dv[i] for i in np.arange(T)]) + rewards[5]

prob += dv[3] >= lpSum([new_mat[6, i] * dv[i] for i in np.arange(T)]) + rewards[6]
prob += dv[3] >= lpSum([new_mat[7, i] * dv[i] for i in np.arange(T)]) + rewards[7]
prob += dv[3] >= lpSum([new_mat[8, i] * dv[i] for i in np.arange(T)]) + rewards[8]


prob

mdp01:
MINIMIZE
None
SUBJECT TO
_C1: 0.24 dv_0 - 0.095 dv_1 - 0.095 dv_2 >= 20000

_C2: 0.335 dv_1 - 0.19 dv_2 - 0.095 dv_3 >= 16000

_C3: - 0.76 dv_0 + 0.905 dv_1 - 0.095 dv_2 >= 10000

_C4: 0.335 dv_2 - 0.285 dv_3 >= 12000

_C5: - 0.665 dv_1 + 0.81 dv_2 - 0.095 dv_3 >= 14000

_C6: - 0.76 dv_0 - 0.095 dv_1 + 0.905 dv_2 >= 10000

_C7: 0.05 dv_3 >= 5000

_C8: - 0.665 dv_1 - 0.19 dv_2 + 0.905 dv_3 >= 14000

_C9: - 0.76 dv_0 - 0.095 dv_1 - 0.095 dv_2 + dv_3 >= 10000

VARIABLES
dv_0 Continuous
dv_1 Continuous
dv_2 Continuous
dv_3 Continuous

In [183]:

# Objective function
prob += dv[0] + dv[1] + dv[2] + dv[3], "Objective"
    
    
prob.writeLP("mdp01.lp")
    
status = prob.solve(GLPK(options=["--ranges","mdp01.sen"]))
print(status)

#print the result
print("dv")
for i in range(0, T):
    print(dv[i].value())
    
print("Objective", value(prob.objective))

1
dv
362000.0
352000.0
352000.0
352000.0
Objective 1418000.0


In [ ]:
# %load mdp01.sen
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 1418000 (MINimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 _C1          NL   20000.00000        .        20000.00000    17894.73684     -61.80000   1.28789e+06 _C2
                                          61.80000          +Inf           +Inf          +Inf          +Inf

     2 _C2          BS   17600.00000   -1600.00000   16000.00000    20315.78947     -20.00000     1.066e+06 _C3
                                            .               +Inf    15733.33333      10.52632   1.60326e+06 _C9

     3 _C3          NL   10000.00000        .        10000.00000     6279.06977      -8.60000     1.386e+06 _C2
                                           8.60000          +Inf    16315.78947          +Inf   1.47232e+06 _C5

     4 _C4          BS   17600.00000   -5600.00000   12000.00000    24842.10526     -20.00000     1.066e+06 _C6
                                            .               +Inf    12800.00000       3.50877   1.47975e+06 _C9

     5 _C5          BS   17600.00000   -3600.00000   14000.00000    32842.10526      -9.50276   1.25075e+06 _C6
                                            .               +Inf    16000.00000      10.52632   1.60326e+06 _C9

     6 _C6          NL   10000.00000        .        10000.00000     6022.09945      -8.60000   1.38379e+06 _C5
                                           8.60000          +Inf    26842.10526          +Inf   1.56284e+06 _C2

     7 _C7          BS   17600.00000  -12600.00000    5000.00000    18442.10526     -20.00000     1.066e+06 _C9
                                            .               +Inf    17600.00000          +Inf          +Inf

     8 _C8          BS   17600.00000   -3600.00000   14000.00000    32842.10526      -1.10497   1.39855e+06 _C9
                                            .               +Inf    14000.00000      15.08772   1.68354e+06 _C3

     9 _C9          NL   10000.00000        .        10000.00000     6022.09945      -1.00000   1.41402e+06 _C8
                                           1.00000          +Inf    26842.10526          +Inf   1.43484e+06 _C2

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 1418000 (MINimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 dv_0         BS  362000.00000       1.00000        .                +Inf      -2.81481   37037.03704 _C1
                                            .               +Inf   362000.00000          +Inf          +Inf

     2 dv_1         BS  352000.00000       1.00000        .        370315.78947      -1.96552  374137.93103 _C3
                                            .               +Inf   352000.00000          +Inf          +Inf

     3 dv_2         BS  352000.00000       1.00000        .        400842.10526      -1.96552  374137.93103 _C6
                                            .               +Inf   352000.00000          +Inf          +Inf

     4 dv_3         BS  352000.00000       1.00000        .        368842.10526        .          1.066e+06 _C9
                                            .               +Inf   352000.00000          +Inf          +Inf

End of report


# Finite Timeframe - By Dynamic Programming - Not finding any way to programatically do this. Need to look further
- We look at four periods for this period
- Best policy after 4 periods

In [43]:
policy_state_transition = [[(0,0,0)], 
                           [(1,0,1), (1,2,0)], 
                           [(2,0,2), (2,1,1), (2,2,0)], 
                           [(3,0,3), (3,1,1), (3,2,0)]]

v5 = [0,0,0,0]
v4 = v5.copy()

def get_stage_rewards(policy_state, reward):
    rewards = []
    for p in policy_state:
        rewards.append(state_action_imm_rew[p[0],p[1]])
    print(rewards)
    return rewards

def get_val_per_stage():
    maxVal = 0
    best_policy = []
    x1 = policy_state_transition[0]
    x2 = policy_state_transition[1]
    x3 = policy_state_transition[2]
    x4 = policy_state_transition[3]

    for i in range(len(x1)):
        for j in range(len(x2)):
            for k in range(len(x3)):
                for l in range(len(x4)):
                    policy = []

                    policy.append(x1[i])
                    policy.append(x2[j])
                    policy.append(x3[k])
                    policy.append(x4[l])
                    value_fn = get_stage_rewards(policy,state_action_imm_rew)
                    print(sum(value_fn))

                    if (sum(value_fn) > maxVal):
                        maxVal = sum(value_fn)
                        best_policy = policy
    return (best_policy, maxVal)

[array([20000.]), array([16000.]), array([12000.]), array([5000.])]
[53000.]
[array([20000.]), array([16000.]), array([12000.]), array([14000.])]
[62000.]
[array([20000.]), array([16000.]), array([12000.]), array([10000.])]
[58000.]
[array([20000.]), array([16000.]), array([14000.]), array([5000.])]
[55000.]
[array([20000.]), array([16000.]), array([14000.]), array([14000.])]
[64000.]
[array([20000.]), array([16000.]), array([14000.]), array([10000.])]
[60000.]
[array([20000.]), array([16000.]), array([10000.]), array([5000.])]
[51000.]
[array([20000.]), array([16000.]), array([10000.]), array([14000.])]
[60000.]
[array([20000.]), array([16000.]), array([10000.]), array([10000.])]
[56000.]
[array([20000.]), array([10000.]), array([12000.]), array([5000.])]
[47000.]
[array([20000.]), array([10000.]), array([12000.]), array([14000.])]
[56000.]
[array([20000.]), array([10000.]), array([12000.]), array([10000.])]
[52000.]
[array([20000.]), array([10000.]), array([14000.]), array([5000.])]


[(0, 0, 0), (1, 0, 1), (2, 1, 1), (3, 1, 1)]